In [42]:
import pandas as pd
from unicodedata2 import normalize

In [43]:
def normalizar_colunas(dataframe):
    lista_colunas = []
    for coluna in dataframe.columns:
        nova_coluna = normalize('NFKD', coluna).encode('ascii', 'ignore').decode('utf-8')#.replace(' ', '_')
        lista_colunas.append(nova_coluna)
    dataframe.columns = lista_colunas
    return dataframe

def salvamento_csv(dataframe, file_name):
    dataframe.to_csv(f"..\\CSV\\01.bronze\\bronze_{file_name}.csv", index=False)

In [44]:
arquivo_completo = pd.read_excel(r"..\Excel\Sumula_Teste_T01B01.xlsx", sheet_name=None, engine="calamine")

df_individual = arquivo_completo["Individual"]
normalizar_colunas(df_individual)

df_times = arquivo_completo["Times"]
normalizar_colunas(df_times)

df_info = arquivo_completo["Informações"]
normalizar_colunas(df_info)


salvamento_csv(df_individual, "desempenho_individual")
salvamento_csv(df_times, "desempenho_times.csv")
salvamento_csv(df_info, "info_partidas")


In [45]:
df_individual

,Baba,Jogador,Time,Gols,Assistencias
0,T01B01,José,Vermelho,8,3
1,T01B01,Renato,Vermelho,2,4
2,T01B01,Matheus,Vermelho,0,2
3,T01B01,Arthur,Vermelho,1,1
4,T01B01,Heitor,Vermelho,1,0
5,T01B01,Isaac,Azul,3,0
6,T01B01,Nicolas,Azul,0,3
7,T01B01,Murilo,Azul,1,1
8,T01B01,Vicente,Azul,0,0
9,T01B01,Paulo,Azul,1,1


In [46]:
df_times

,Baba,Time,Campeao,Vitorias,Empates,Derrotas
0,T01B01,Vermelho,1,5,2,1
1,T01B01,Azul,0,2,3,3
2,T01B01,Verde,0,1,3,4


In [47]:
df_info

,Baba,Data,Inicio,Fim,Arbitro,Nota,Campo,Coordenadas
0,T01B01,2024-08-17,16:00:00,19:00:00,Carlinho Apito Longo,8,Complexo Poliesportivo da UEFS,"-12.197253215204276, -38.97078135645937"
